In [ ]:
import urllib
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import openapi

: 

In [ ]:
import requests

# ✅ API 키 및 URL 설정
s_key = 'PwfNqgN7hal2oSaDMz96GOgjRcDcPrZUybaows5KKV1Xa0uMKVhAbb/e2y78IJMwQ+mHPTsZgjBVqkk54niRUA=='
url = 'http://apis.data.go.kr/B551172/Kidney11/kiCaseByType'

# ✅ 기본 파라미터 설정 (numOfRows를 정수로 변경)
params = {
    'serviceKey': s_key,
    'type': 'json',
    'numOfRows': 100,  # 🔹 문자열이 아닌 정수로 변경
    'centerNm': '국립암센터',
    'fromYear': '2010',
    'toYear': '2019',
    'pageNo': 1
}

all_data = []

# 1️⃣ 먼저 첫 번째 요청을 보내서 totalCount 확인
response = requests.get(url, params=params)

if response.status_code == 200:
    resp_dict = response.json()

    # 🔹 totalCount가 없을 경우 기본값 0으로 설정 (오류 방지)
    total_count = int(resp_dict.get('totalCount', 0) or 0)  
    items_per_page = params['numOfRows']  
    total_pages = (total_count // items_per_page) + (1 if total_count % items_per_page else 0)

    print(f"📌 총 데이터 개수: {total_count}, 가져올 페이지 수: {total_pages}")

    # ✅ 데이터가 없을 경우 메시지 출력 후 종료
    if total_count == 0:
        print("⚠️ 가져올 데이터가 없습니다.")
    else:
        # 2️⃣ total_pages 만큼 반복하여 모든 데이터를 가져오기
        for i in range(1, total_pages + 1):
            page_params = params.copy()  # 🔹 원본 params 보호
            page_params['pageNo'] = i  

            response = requests.get(url, params=page_params)

            if response.status_code == 200:
                resp_dict = response.json()
                items = resp_dict.get('items', [])  # 🔹 데이터가 없을 경우 빈 리스트 반환
                all_data.extend(items)
                print(f"📄 페이지 {i}/{total_pages} 처리 완료! (누적 {len(all_data)}개)")
            else:
                print(f"❌ 데이터 요청 실패 (페이지 {i})")

        print(f"✅ 총 {len(all_data)}개의 데이터 수집 완료!")

else:
    print(f"❌ API 요청 실패 - HTTP 상태 코드: {response.status_code}")


df_past = pd.DataFrame(all_data)
df_past

In [ ]:
df_past = df_past.iloc[:, [1,2,3,4,5,-2]]

In [ ]:
df_past.rename(columns={'centerNm':'센터 이름', 'critYr':'기준 년도', 'statsTrgtNm':'고혈압 병력 여부', 'ptAge':'연령' ,'ptSexCd':'성별', 'statsTrgtNm':'암 발병 이력 보유 여부', 'ptCntNmvl': '환자수'}, inplace=True)
df_past

In [ ]:
df_past['환자수'] = pd.to_numeric(df_past['환자수'], errors='coerce')

df_past['환자수'] = df_past['환자수'].fillna(0).astype(int)

past_count = df_past.groupby(df_past['암 발병 이력 보유 여부'].fillna('무응답'))['환자수'].sum()

print(past_count)

In [ ]:
import matplotlib.font_manager as fm

# 한글 폰트 설정 (예: 맑은 고딕)
plt.rcParams['font.family'] = 'Malgun Gothic'

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 파이차트 그리기
plt.figure(figsize=(6, 6))
plt.pie(past_count.values, labels=past_count.index, autopct='%1.1f%%', startangle=140)
plt.title('암 발병 이력 보유 여부 분포')
plt.axis('equal')
plt.show()